In [72]:
from openpyxl import load_workbook
import re

wb = load_workbook(r'E:\pharmacodynamics\sample data\summary table\DAR Ki values 34485-494.xlsx', data_only=True)


In [73]:
for name in wb.sheetnames:
    sheet=wb[name]
    
    print(sheet)

<Worksheet "D1">
<Worksheet "D2 Ki">
<Worksheet "D3 Ki">
<Worksheet "D4.4 Ki">
<Worksheet "Clean sheet">


In [89]:
def find_header(row, header):
    indices = [i for i, item in enumerate(row) if isinstance(item, str) and re.search(header, item, re.IGNORECASE)]
    return indices

keyHeaders = ['ic50', 'ki', 'hill slope']
valHeaders = ['ave', 'sem']

sheet1=wb["D1"]
matchstr = r"\b(" + "|".join(map(re.escape, keyHeaders)) + r")\b"
print(matchstr)
#matchstr = re.compile('ave')
for idx, row in enumerate(sheet1.iter_rows(values_only=True)):
    ic50 = find_header(row, "ic50")
    ki = find_header(row, "ki")
    hillSlope = find_header(row, "hill slope")
    averages = find_header(row, "ave")
    sem = find_header(row, "sem")
    headerRow = find_header(row, matchstr)
    if headerRow:
        drugID = row[0]
        ic50Average = sheet1.cell(row=idx+2, column=ic50[0]+2).value
        ic50SEM = sheet1.cell(row=idx+2, column=ic50[0]+3).value
        kiAverage = sheet1.cell(row=idx+2, column=ki[0]+2).value
        kiSEM = sheet1.cell(row=idx+2, column=ki[0]+3).value
        hillAverage = sheet1.cell(row=idx+2, column=hillSlope[0]+2).value
        hillSEM = sheet1.cell(row=idx+2, column=hillSlope[0]+3).value
        print(drugID)
        print(ic50Average)
        print(ic50SEM)
        print(kiAverage)
        print(kiSEM)
        print(hillAverage)
        print(hillSEM)

\b(ic50|ki|hill\ slope)\b
SCH-23390
1.0019666666666667
0.05876394397171715
0.7205645996587119
0.044974034674686296
-0.9764333333333335
0.06478339121856606
SKF-38393
395.1666666666667
59.085371379988096
284.00355608999234
42.149586145380624
-0.7035666666666667
0.04326254474459147
34485
3393
302.2339050029519
2438.600707787093
214.62042268143983
-1.1120333333333334
0.056855674983046126
34486
6535.666666666667
209.46784425724582
4697.52680795471
143.6332852724657
-1.3393333333333335
0.06690872721684207
34487
11866.666666666666
95.27737285304303
8530.059905310854
77.45659845749131
-1.345
0.0901794507265005
34488
605.6999999999999
18.520889107527577
435.4569100221253
14.395839414512738
-1.2910000000000001
0.1107173578682825
34489
1101.4333333333334
101.46947543199525
792.2560421250136
75.94419948669126
-1.24
0.09859175083815865
34490
7109.666666666667
499.5305574014244
5107.3283787743585
337.32366895759645
-1.3583333333333334
0.11305505050392237
34491
8927.333333333334
699.2687450313908
642